# 初めに

数独を焼きなましで解いてみよう！のこぼれ話です。数独の盤面を受け取り、スコアを返す関数をいくつか作成しました。スコアは同じ行・列・ボックスに同じ数字が複数含まれていると高くなり、焼きなましではそのスコアを低くすることにより最適解を探します。スコアを求める方法をいくつか作り、その時に得た高速化の知見をメモします。なお、スコアを求めるには盤面の更新によるスコアの差分を取るほうが速いため、以下の方法は実際にはあまり使いません。

数独の盤面はMatrixで表現しました。スコアの計算方法は次の通りです。  
$$ score = \sum_{i=1}^{9} [(9 - 列iに含まれる数の種類) + (9 - 行iに含まれる数の種類) + (9 - ボックスiに含まれる数の種類)] $$

# 準備

In [1]:
using Random
using BenchmarkTools

In [2]:
board = Random.rand(1:9, 9, 9)

9×9 Matrix{Int64}:
 3  7  7  2  5  3  1  9  4
 1  2  2  9  7  1  9  4  8
 1  7  4  4  5  3  9  1  3
 4  1  1  6  1  2  7  7  5
 3  6  2  8  9  7  6  4  3
 7  2  4  7  9  3  7  8  2
 2  8  6  8  6  7  9  8  4
 2  4  9  9  6  6  2  8  5
 8  9  4  3  8  2  9  2  7

# 方法1:Set()を使う

まずは定義どおりに実装します。同じ列・行・ボックスに含まれる数字の種類は`Set()`を使うことで数えました。後から分かったことですが、`Set()`の処理にかなり時間がかかります。

In [3]:
function evaluate1(board)
    score = 0
    for i in 1:9
        score += (9 - length(Set(board[i,:]))) #行i
        score += (9 - length(Set(board[:,i]))) #列i
        score += (9 - length(Set(board[(i-1)%3 * 3 + 1 : (i-1)%3 * 3 + 3, div(i-1, 3) * 3 + 1 : div(i-1, 3) * 3 + 3])))
    end
    return score
end

evaluate1 (generic function with 1 method)

In [4]:
evaluate1(board)

79

In [5]:
@benchmark evaluate1(board)

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.800 μs … 298.550 μs  ┊ GC (min … max): 0.00% … 97.16%
 Time  (median):     5.983 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.633 μs ±  11.618 μs  ┊ GC (mean ± σ):  7.34% ±  4.10%

  ▃▇██▇▄▄▄▅▃▂▁                           ▁                    ▂
  █████████████▇▅▆▇▅▇▅▄▃▃▄▁▄▁▅▁▄▃▃▁▅▄▆▇███████▇▇█▇▇▆▆▇▅▇▇▆▆▆▇ █
  5.8 μs       Histogram: log(frequency) by time      8.98 μs <

 Memory estimate: 13.92 KiB, allocs estimate: 135.

In [6]:
print(evaluate1(board))

79

# 方法2:登場した数字の種類を配列で管理する

次に配列を使って登場した数字の種類を数えます。`nums`は要素数9の配列で、初期値はすべて0です。7が登場したら、`nums[7] = 1`とすることにより、登場した数字がどれなのか管理します。配列の要素の型をInt64からInt8にすると9%ほど高速になりました。Bool型を使うともっと速くなるのかな？

In [7]:
function evaluate2(board)
    score = 0
    for i in 1:9
        tmp_h = zeros(Int64, 9)
        tmp_v = zeros(Int64, 9)
        tmp_box = zeros(Int64, 9)
        for j in 1:9
            tmp_h[board[i,j]] = 1
            tmp_v[board[j,i]] = 1
            tmp_box[board[((i-1)%3)*3 + (j-1)%3 + 1, div(i-1, 3)*3 + div(j-1, 3) + 1]] = 1
        end
        score += (27 - sum(tmp_h) - sum(tmp_v) - sum(tmp_box))
    end
    return score
end

evaluate2 (generic function with 1 method)

In [8]:
evaluate2(board)

79

In [9]:
@benchmark evaluate2(board)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.220 μs … 43.230 μs  ┊ GC (min … max): 0.00% … 93.50%
 Time  (median):     1.290 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.438 μs ±  1.121 μs  ┊ GC (mean ± σ):  1.91% ±  2.47%

   ▆█▇▆▄▂  ▁▂▂▁                        ▂▃▂▃▂▂▂▁▁▁▁           ▂
  ▆█████████████▇▆▅▅▄▃▃▃▃▅▁▃▃▁▁▃▁▃▁▁▄▃█████████████▇▇▆▅▆▅▅▅▅ █
  1.22 μs      Histogram: log(frequency) by time     2.43 μs <

 Memory estimate: 3.38 KiB, allocs estimate: 27.

# 方法3:方法2の改良版。配列の生成回数を減らす

方法2の改良版です。イメージとしては方法2で9回生成していた配列を1つに繋げたというものです。結構高速になりますね。可読性が怪しくなるのが欠点か。

In [10]:
function evaluate3(board)
    tmp_h = zeros(Int8, 9*9)
    tmp_v = zeros(Int8, 9*9)
    tmp_box = zeros(Int8, 9*9)
    for i in 1:9
        for j in 1:9
            tmp_h[board[i,j] + (i-1)*9] = 1
            tmp_v[board[j,i] + (i-1)*9] = 1
            tmp_box[board[((i-1)%3)*3 + (j-1)%3 + 1, div(i-1, 3)*3 + div(j-1, 3) + 1] + (i-1)*9] = 1
        end
    end
    return (9*9*3 - sum(tmp_h) - sum(tmp_v) - sum(tmp_box))
end

evaluate3 (generic function with 1 method)

In [11]:
evaluate3(board)

79

In [12]:
@benchmark evaluate3(board)

BenchmarkTools.Trial: 10000 samples with 196 evaluations.
 Range (min … max):  467.857 ns …  2.797 μs  ┊ GC (min … max): 0.00% … 71.20%
 Time  (median):     485.714 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   498.498 ns ± 94.595 ns  ┊ GC (mean ± σ):  0.69% ±  3.25%

  ▅▇▆██▇▆▃▁                 ▂▃▂▂▂                              ▂
  ██████████▇▆▆▅▆▆▄▇▇▅▆▇▆▆▅████████▆▄▅▄▄▄▄▅▅▅▅▆▅▅▄▅▁▄▅▄▃▄▄▄▄▄▄ █
  468 ns        Histogram: log(frequency) by time       722 ns <

 Memory estimate: 432 bytes, allocs estimate: 3.

# 方法4:方法3からさらに配列をまとめる

方法3では3つの配列を使っていましたが、それらを1つに繋げました。ここまでくると可読性は壊滅的ですね。

In [13]:
function evaluate4(board)
    tmp = zeros(Int8, 81*3)
    for i in 1:9
        for j in 1:9
            tmp[board[i,j] + (i-1)*9] = 1
            tmp[board[j,i] + (i-1)*9 + 81] = 1
            tmp[board[((i-1)%3)*3 + (j-1)%3 + 1, div(i-1, 3)*3 + div(j-1, 3) + 1] + (i-1)*9 + 81*2] = 1
        end
    end
    return (9*9*3 - sum(tmp))
end

evaluate4 (generic function with 1 method)

In [14]:
evaluate4(board)

79

In [15]:
@benchmark evaluate4(board)

BenchmarkTools.Trial: 10000 samples with 208 evaluations.
 Range (min … max):  369.712 ns …  2.511 μs  ┊ GC (min … max): 0.00% … 80.95%
 Time  (median):     383.654 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   396.258 ns ± 82.138 ns  ┊ GC (mean ± σ):  0.64% ±  2.90%

  ▃▅▂▇█▅▆▅▃▂               ▁▃▂▁▁▁                              ▂
  ███████████▇▇▆▅▄▆▅▇▆▆▆▅▇█████████▇██▇▇▅▆▆▆▄▅▅▄▅▄▃▅▆▃▁▅▄▅▅▄▄▅ █
  370 ns        Histogram: log(frequency) by time       549 ns <

 Memory estimate: 304 bytes, allocs estimate: 1.

# まとめ

今回の実験で分かったこと
1. `Set()`は時間がかかり、今回のように限られた範囲の値を扱うなら配列のほうが便利かも
2. 0 or 1　のように取りうる値の範囲が限られている場合、Int64よりInt8などを使う方が速い
3. 配列の生成回数は少なくしよう。（`sum()`に時間がかかっているだけかも？）　　


|方法 |ベンチマーク結果|備考|
|:---:|:------------:  |:---|
|方法1|6.633 μs ±  11.618 μs|Set()を使う|
|方法2|1.438 μs ±  1.121 μs|登場した数字の種類を配列で管理する|
|方法3|498.498 ns ± 94.595 ns|方法2の改良版。配列の生成回数を減らす|
|方法4|396.258 ns ± 82.138 ns|方法3からさらに配列をまとめる|